## Megan Hoeksema

## Assignment 5: Naive Bayes

#### Packages

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.naive_bayes import CategoricalNB, GaussianNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import string

### Question 1

#### 1A. Upload the income_evaluation_cat.csv provided on canvas.  The features in this data include workclass, education, race, and gender. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to 50,000 or greater than 50,000 respectively. Print the unique values of each variable in this data. 

In [4]:
incomeCat = pd.read_csv('income_evaluation_cat.csv', skipinitialspace = True)
incomeCat

,workclass,education,race,gender,income
0,State-gov,Bachelors,White,Male,<=50K
1,Self-emp-not-inc,Bachelors,White,Male,<=50K
2,Private,HS-grad,White,Male,<=50K
3,Private,11th,Black,Male,<=50K
4,Private,Bachelors,Black,Female,<=50K
...,...,...,...,...,...
32556,Private,Assoc-acdm,White,Female,<=50K
32557,Private,HS-grad,White,Male,>50K
32558,Private,HS-grad,White,Female,<=50K
32559,Private,HS-grad,White,Male,<=50K


In [5]:
for col in incomeCat:
  print(incomeCat[col].unique())

['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
['Male' 'Female']
['<=50K' '>50K']


#### 1B. You will implement Naïve Bayes from scratch using Bayes’ rule. You can do your calculations in Python, but you would not use the sklearn package. Suppose that all the income_evaluation_cat.csv data you uploaded is the training data , classify a test instance, X = [“Private”, “Bachelors”, “White”, “Female”] into the class income<=50 or income>50k. Compute prior and posterior probabilities that are needed to classify the test instance. 

#### What are the prior probabilities, P(income<=50) and P(income>50k)?

In [6]:
p_less_50 = (incomeCat['income'].value_counts()['<=50K'])/len(incomeCat)
print('p(<=50K): ' + str(p_less_50))

p_great_50 = (incomeCat['income'].value_counts()['>50K'])/len(incomeCat)
print('p(>50K): ' + str(p_great_50))

p(<=50K): 0.7591904425539756
p(>50K): 0.2408095574460244


In [7]:
p_x = (sum((incomeCat['workclass'] == 'Private') & (incomeCat['education'] == 'Bachelors') 
        & (incomeCat['race'] == 'White') & (incomeCat['gender'] == 'Female')))/len(incomeCat)
p_x

0.028193237308436474

In [8]:
# (p(x|<=50k) = p(x and <=50k)/p(x)

p_x_less = (sum((incomeCat['workclass'] == 'Private') & (incomeCat['education'] == 'Bachelors') 
        & (incomeCat['race'] == 'White') & (incomeCat['gender'] == 'Female') & (incomeCat['income'] == '<=50K')))/len(incomeCat)

p_x_less_x = p_x_less/p_x

p_x_less_x

0.7952069716775598

In [9]:
# (p(x|>50k) = p(x and >50K)/p(x)

p_x_great = (sum((incomeCat['workclass'] == 'Private') & (incomeCat['education'] == 'Bachelors') 
        & (incomeCat['race'] == 'White') & (incomeCat['gender'] == 'Female') & (incomeCat['income'] == '>50K')))/len(incomeCat)

p_x_great_x = p_x_great/p_x

p_x_great_x

0.2047930283224401

#### What are the posterior probabilities, P(income<=50/X) and P(income>50k/X)?

In [10]:
# posterior = (likelihood * prior)/evidence

# p(<=50K|x) = (p(x|<=50k) * p(<=50K)) / p(x)
print('P(<=50K|X): ' + str((p_less_50 * p_x) / ((p_less_50 * p_x_less_x)+(p_great_50 * p_x_great_x))))
      
#p(>50k|x) = (p(x|>50k) * p(>50k)) / p(x)
print('P(>50K|X): ' + str((p_great_50 * p_x) / ((p_less_50 * p_x_less_x)+(p_great_50 * p_x_great_x))))

P(<=50K|X): 0.03277651522814054
P(>50K|X): 0.0103964666627771


#### Based on your posterior probabilities, which class should the test instance be classified into? 

The test instance would be classified into less than or equal to 50K

#### 1C. Preprocess or transform the features in the income_evaluation_cat.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format. 

In [11]:
incomeCat2 = incomeCat

incomeCat2['workclass'].replace(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov', 'Local-gov', '?', 
                                 'Self-emp-inc', 'Without-pay', 'Never-worked'],[0, 1, 2, 3, 4, 5, 6, 7, 8], inplace = True)
incomeCat2['education'].replace(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', 
                                 '7th-8th', 'Doctorate', 'Prof-school', '5th-6th', '10th', '1st-4th', 'Preschool', '12th'], 
                                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], inplace = True)
incomeCat2['race'].replace(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'], [0, 1, 2, 3, 4], 
                           inplace = True)
incomeCat2['gender'].replace(['Male', 'Female'], [0, 1], inplace = True)

incomeCat2

,workclass,education,race,gender,income
0,0,0,0,0,<=50K
1,1,0,0,0,<=50K
2,2,1,0,0,<=50K
3,2,2,1,0,<=50K
4,2,0,1,1,<=50K
...,...,...,...,...,...
32556,2,6,0,1,<=50K
32557,2,1,0,0,>50K
32558,2,1,0,1,<=50K
32559,2,1,0,0,<=50K


In [12]:
scaler = MinMaxScaler()
contCols = ['education', 'workclass', 'race', 'gender']
contCols
incomeCat2.loc[:,contCols] = scaler.fit_transform(incomeCat2.loc[:,contCols])
incomeCat2

,workclass,education,race,gender,income
0,0.000,0.000000,0.00,0.0,<=50K
1,0.125,0.000000,0.00,0.0,<=50K
2,0.250,0.066667,0.00,0.0,<=50K
3,0.250,0.133333,0.25,0.0,<=50K
4,0.250,0.000000,0.25,1.0,<=50K
...,...,...,...,...,...
32556,0.250,0.400000,0.00,1.0,<=50K
32557,0.250,0.066667,0.00,0.0,>50K
32558,0.250,0.066667,0.00,1.0,<=50K
32559,0.250,0.066667,0.00,0.0,<=50K


#### 1D. Randomly split the transformed input data and the output data into X_train, y_train, X_test and y_test using tools in sklearn. 

In [13]:
X = incomeCat2.drop(columns = ['income'])
y = incomeCat2['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

print("Sizes of training and test sets:", [len(elem) for elem in [X_train, X_test, y_train, y_test]])

Sizes of training and test sets: [22792, 9769, 22792, 9769]


#### 1E. Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set. 

In [14]:
cnb = CategoricalNB()
y_pred = cnb.fit(X_train, y_train).predict(X_test)
accuracy_score(y_pred, y_test)

0.7631282628723514

### Question 2

#### 2A. Upload the income_evaluation_continuous.csv data provided on canvas.  The features in this data include age, education_num, and hours_per_week. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to 50,000 or greater than 50,000 respectively. Compute the mean and standard deviation of each input variable such that the results are presented on the same table or data frame. You can call the .apply() function on the pandas DataFrame.  

In [15]:
incomeCont = pd.read_csv('income_evaluation_continuous.csv', skipinitialspace = True)
incomeCont

,age,education_num,hours_per_week,income
0,39,13,40,<=50K
1,50,13,13,<=50K
2,38,9,40,<=50K
3,53,7,40,<=50K
4,28,13,40,<=50K
...,...,...,...,...
32556,27,12,38,<=50K
32557,40,9,40,>50K
32558,58,9,40,<=50K
32559,22,9,20,<=50K


In [16]:
incomeCont.agg([np.mean, np.median, np.std])

/tmp/ipykernel_587/731884038.py:1: FutureWarning: ['income'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  incomeCont.agg([np.mean, np.median, np.std])


,age,education_num,hours_per_week
mean,38.581647,10.080679,40.437456
median,37.000000,10.000000,40.000000
std,13.640433,2.572720,12.347429


#### 2B. You will implement Naïve Bayes from scratch using Bayes’ rule. Assume that all the features or input variables follow a normal distribution. You can do your calculations in Python, but you would not use the sklearn package. You can use the density function inside the stats module in the SciPy package. Given that all the income_evaluation_continuous.csv data you uploaded is the training data, classify a test instance, X = [30, 10, 45], into the class income<=50 or income>50k. Compute prior and posterior probabilities that are needed to classify the test instance. 

#### What are the prior probabilities, P(income<=50) and P(income>50k)?

In [17]:
p_less_50 = (incomeCont['income'].value_counts()['<=50K'])/len(incomeCont)
print('p(<=50K): ' + str(p_less_50))

p_great_50 = (incomeCont['income'].value_counts()['>50K'])/len(incomeCont)
print('p(>50K): ' + str(p_great_50))

p(<=50K): 0.7591904425539756
p(>50K): 0.2408095574460244


In [18]:
p_x = (sum((incomeCont['age'] == 30) & (incomeCont['education_num'] == 10) 
        & (incomeCont['hours_per_week'] == 45)))/len(incomeCont)
p_x

0.0003685390497834833

In [19]:
# (p(x|<=50k) = p(x and <=50k)/p(x)

p_x_less = (sum((incomeCont['age'] == 30) & (incomeCont['education_num'] == 10) 
        & (incomeCont['hours_per_week'] == 45) & (incomeCat['income'] == '<=50K')))/len(incomeCat)

p_x_less_x = p_x_less/p_x

p_x_less_x

0.8333333333333334

In [20]:
# (p(x|>50k) = p(x and >50K)/p(x)

p_x_great = (sum((incomeCont['age'] == 30) & (incomeCont['education_num'] == 10) 
        & (incomeCont['hours_per_week'] == 45) & (incomeCat['income'] == '>50K')))/len(incomeCat)

p_x_great_x = p_x_great/p_x

p_x_great_x

0.16666666666666666

#### What are the posterior probabilities, P(income<=50/X) and P(income>50k/X)?

In [21]:
# posterior = (likelihood * prior)/evidence

# p(<=50K|x) = (p(x|<=50k) * p(<=50K)) / p(x)
print('P(<=50K|X): ' + str((p_less_50 * p_x) / ((p_less_50 * p_x_less_x)+(p_great_50 * p_x_great_x))))
      
#p(>50k|x) = (p(x|>50k) * p(>50k)) / p(x)
print('P(>50K|X): ' + str((p_great_50 * p_x) / ((p_less_50 * p_x_less_x)+(p_great_50 * p_x_great_x))))

P(<=50K|X): 0.00041586500303471713
P(>50K|X): 0.0001319092835273146


#### Based on your posterior probabilities, which class should the test instance be classified into? 

The test instance would be classified into less than or equal to 50K

#### 2C. Preprocess or transform the features in the income_evaluation_cont.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format. 

In [23]:
scaler = MinMaxScaler()
contCols = ['age', 'education_num', 'hours_per_week']
contCols
incomeCont.loc[:,contCols] = scaler.fit_transform(incomeCont.loc[:,contCols])
incomeCont

,age,education_num,hours_per_week,income
0,0.301370,0.800000,0.397959,<=50K
1,0.452055,0.800000,0.122449,<=50K
2,0.287671,0.533333,0.397959,<=50K
3,0.493151,0.400000,0.397959,<=50K
4,0.150685,0.800000,0.397959,<=50K
...,...,...,...,...
32556,0.136986,0.733333,0.377551,<=50K
32557,0.315068,0.533333,0.397959,>50K
32558,0.561644,0.533333,0.397959,<=50K
32559,0.068493,0.533333,0.193878,<=50K


#### 2D. Randomly split the input and output data into X_train, y_train, X_test and y_test using tools in sklearn. 

In [24]:
X = incomeCont.drop(columns = ['income'])
y = incomeCont['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

print("Sizes of training and test sets:", [len(elem) for elem in [X_train, X_test, y_train, y_test]])

Sizes of training and test sets: [22792, 9769, 22792, 9769]


#### 2E. Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set. 

In [25]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
accuracy_score(y_pred, y_test)

0.7988535162247927

### Question 3

#### 3A. Upload the True.csv data provided on canvas into Python. You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “True”. So, your new data frame should have three columns; “title”, “text” and “news_type”. 

In [26]:
true = pd.read_csv('True.csv')
trueDF = true.drop(columns = ['subject', 'date'])
trueDF['news_type'] = 'True'
trueDF

,title,text,news_type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,True
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,True
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",True
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,True
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,True


#### 3B. Upload the Fake.csv data provided on canvas into Python. You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “Fake”. So, your new data frame should have three columns; “title”, “text” and “news_type”.

In [27]:
fake = pd.read_csv('Fake.csv')
fakeDF = fake.drop(columns = ['subject', 'date'])
fakeDF['news_type'] = 'Fake'
fakeDF

,title,text,news_type
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,Fake
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Fake


#### 3C. Merge the data frame in a) and b) so that one of the data frames is stacked vertically on top of the other. Combine the text in the “title” and “text” columns of the merged data frame into another column called “news”. Drop the “title” and “text” columns so that your final data frame is has only two columns, “news” and “news_type”. Print the first five rows of your final data frame. 

In [28]:
df_merged = pd.concat([trueDF, fakeDF])
df_merged['news'] = df_merged['title'] + df_merged['text']
df_merged = df_merged.drop(columns = ['title', 'text'])
df_merged.head(5)

,news_type,news
0,True,"As U.S. budget fight looms, Republicans flip t..."
1,True,U.S. military to accept transgender recruits o...
2,True,Senior U.S. Republican senator: 'Let Mr. Muell...
3,True,FBI Russia probe helped by Australian diplomat...
4,True,Trump wants Postal Service to charge 'much mor...


#### 3D. Preprocess your data by cleaning the textual data in the “news” column and removing the stop words, special characters, punctuations, etc especially at the beginning and end of each word. You can display any messy news text before you clean the data, then display the messy news text again after cleaning the data to see if your data cleaning worked well. Also, drop instances where the news text is less than 50 words. 

In [29]:
sw = set(stopwords.words("english"))
print(sw)

{'had', 'once', 'was', 'too', 'mustn', 'can', "aren't", 'other', 'not', 'have', 'that', 'shan', 'mightn', 'or', 'be', 'does', 'its', 'down', 'is', 'nor', 'an', 'no', 't', 'hers', "hasn't", 'your', 'them', 'at', 'our', 'few', 'so', 'to', 'himself', 'the', 'being', 'then', 'we', 'his', 'out', "shouldn't", 'a', 'theirs', "didn't", 'during', 'now', 'haven', 'some', 'they', "mightn't", 'he', 'what', 'both', "wouldn't", 'same', 'don', 'ma', 'before', 'i', 're', 'but', 'myself', 'been', 'll', 'hasn', 'me', 'all', 'themselves', 'with', 'these', 'him', 'you', 'from', 'here', 'than', 'when', 'own', "couldn't", "isn't", 'hadn', 'while', 'and', 'into', "you'd", 'shouldn', 'ours', 'will', 'against', "needn't", 'about', 'were', "hadn't", 'did', 'only', 'further', 'ain', 'which', "she's", 'more', 'has', 'o', 'if', 'couldn', "wasn't", 'wouldn', 'on', 'their', 'wasn', 's', 'should', 'such', 'any', "weren't", 'again', "mustn't", 'didn', "you've", 'off', 'are', 'most', 've', 'because', 'this', 'very', 'd

In [30]:
print(set(string.punctuation))

{'!', '}', '(', '"', '-', '{', '[', ',', ';', '~', "'", '@', '^', '>', '*', '_', '?', '\\', '#', '$', ')', '`', '<', '=', '.', '/', '+', '|', '&', ']', '%', ':'}


In [31]:
import re
def clean(text,stopwords):
    text = re.sub(r'<[^<>]*>',' ',text)
    text_list = text.split()
    text_words = []
    
    punctuation = set(string.punctuation)
    
    for word in text_list:
        while len(word) > 0 and word[0] in punctuation:
            word = word[1:]
        while len(word) > 0 and word[-1] in punctuation:
            word = word[:-1]
        if len(word) > 0 and "/" not in word:
            if word. lower() not in stopwords:
                text_words.append(word.lower())
        cleaner_text = " ".join(text_words)
    return cleaner_text
            
df_merged["news"] = df_merged["news"].apply(clean, stopwords = sw)

df_merged = df_merged[df_merged["news"].str.len()>50]

df_merged.head()

,news_type,news
0,True,u.s budget fight looms republicans flip fiscal...
1,True,u.s military accept transgender recruits monda...
2,True,senior u.s republican senator let mr mueller j...
3,True,fbi russia probe helped australian diplomat ti...
4,True,trump wants postal service charge much amazon ...


In [32]:
df_merged.shape

(44828, 2)

#### 3E. Transform the input text data into feature vectors where the entries of the feature vectors are term-frequency-inverse-document-frequency. Use the TfidfVectorizer() in sklearn. 

In [33]:
texts = df_merged['news']

tfidf = TfidfVectorizer(ngram_range=(1,2), stop_words="english", min_df=10, max_features = None)

matrix = tfidf.fit_transform(texts)

#### 3F. Split the feature vectors and the output variable into into X_train, y_train, X_test and y_test, you can let the test set be 30% of the entire data. 

In [34]:
y = df_merged['news_type']
X = matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

#### 3G. Fit an appropriate Naïve Bayes model and compute the training and test accuracy of the model. Is there overfitting?

In [35]:
multi = MultinomialNB()

y_pred_test = multi.fit(X_train, y_train).predict(X_test)
print('Accuracy of Test: ' + str(accuracy_score(y_pred_test, y_test)))

y_pred_train = multi.fit(X_train, y_train).predict(X_train)
print('Accuracy of Train: ' + str(accuracy_score(y_pred_train, y_train)))

Accuracy of Test: 0.9518179790318982
Accuracy of Train: 0.9574874916345326


No, there is not overfitting as both the accuracy of the test and training sets are very similar

#### 3H. Fit a Naïve Bayes using cross validation and print the average cross validation score as well as the standard deviation of the cross-validation scores. 

In [36]:
scores = cross_val_score(estimator = MultinomialNB(),X = X_train, y = y_train, cv = 5)
print("Average cross validation score: ", scores.mean())
print("Standard deviation of cross validation scores: ", scores.std())

Average cross validation score:  0.947257658975316
Standard deviation of cross validation scores:  0.0034758255149045197


#### 3I. Select some hypermeters of your choice and tune using the grid search cross validation. Use some other hyperparameters than those used in class examples. 

In [37]:
multi.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [38]:
param_grid = [{'alpha': [0, 1, 10, 100, 1000, 10000, 100000]}]
grid = GridSearchCV(multi, param_grid, cv = 5)
grid.fit(X_train, y_train)
grid.best_params_

/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/mnt/c/users/rainb/desktop/scripts/COMP4447/dst1_env/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric

{'alpha': 0}

In [39]:
grid.score(X_train,y_train)

0.9911724401669907

In [40]:
grid.score(X_test,y_test)

0.9721912409844599